In [1]:
import os
from dotenv import load_dotenv
import weaviate
from weaviate.classes.init import Auth
load_dotenv()


True

In [2]:

def get_weaviate_client():
    weaviate_url = os.environ["WEAVIATE_URL"]
    weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
    )

    print(client.is_ready())
    return client

In [3]:
client = get_weaviate_client()

True


In [4]:
COLLECTION_NAME = "CarManual_BasicRecursive"
from weaviate.classes.config import Configure, Property, DataType
def create_collection(client):
    """Create Weaviate collection for this strategy"""
    if client.collections.exists(COLLECTION_NAME):
        client.collections.delete(COLLECTION_NAME)
    
    # Create new collection
    client.collections.create(
        name=COLLECTION_NAME,
        description="Car manual chunks using basic recursive splitting",
        vectorizer_config=Configure.Vectorizer.none(),  # We provide our own vectors
        properties=[
            Property(
                name="text",
                data_type=DataType.TEXT,
                description="The actual text content of the chunk"
            ),
            Property(
                name="car_model",
                data_type=DataType.TEXT,
                description="Car model name (e.g., 'MG_Astor', 'Tata_Tiago')"
            ),
            Property(
                name="page_number",
                data_type=DataType.INT,
                description="Page number in the PDF"
            ),
            Property(
                name="chunk_index",
                data_type=DataType.INT,
                description="Sequential chunk number within the document"
            ),
            Property(
                name="source_file",
                data_type=DataType.TEXT,
                description="Original PDF filename"
            ),
            Property(
                name="chunking_strategy",
                data_type=DataType.TEXT,
                description="Chunking strategy used"
            )
        ]
    )
    
    print(f"✅ Created collection: {COLLECTION_NAME}")

create_collection(client)

/Users/santhosh/Documents/study_projects/ringcentral_assessment/sentence_transformer/lib/python3.10/site-packages/weaviate/warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(


✅ Created collection: CarManual_BasicRecursive


In [5]:
from sentence_transformers import SentenceTransformer
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
model = SentenceTransformer(EMBEDDING_MODEL)

/Users/santhosh/Documents/study_projects/ringcentral_assessment/sentence_transformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model.encode("hi, hello")

array([-5.34153767e-02,  3.65626179e-02,  4.21999246e-02,  5.01070395e-02,
       -6.55106902e-02, -7.78661147e-02,  5.01934476e-02,  3.58336754e-02,
       -4.28080820e-02, -5.05087003e-02,  7.62659404e-03,  3.67246531e-02,
       -2.47361977e-02, -6.82701766e-02,  4.71939705e-02, -1.48678431e-02,
        2.02000532e-02, -1.63237993e-02, -1.56004786e-01, -3.01015121e-03,
       -1.56277847e-02,  1.20060286e-02, -4.48601358e-02,  3.84991951e-02,
       -6.70832247e-02, -7.56756067e-02,  5.27825654e-02,  7.26806596e-02,
       -3.48465294e-02, -8.06678757e-02,  9.05680880e-02,  5.36064282e-02,
        3.27864066e-02,  1.73527319e-02,  2.14876514e-02,  9.40619931e-02,
       -1.01031236e-01, -8.89021382e-02,  1.02828676e-02, -5.79528045e-03,
        2.08677302e-04, -2.64482982e-02,  1.98784005e-03, -1.11314030e-02,
        3.55645902e-02, -3.43850106e-02,  2.42669918e-02,  4.50413637e-02,
        3.50355916e-02,  2.14554332e-02, -9.95509550e-02, -7.01984987e-02,
       -2.27335934e-02, -

In [7]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path="/Users/santhosh/Documents/study_projects/ringcentral_assessment/pdfs/astor_manual.pdf"):
    """Extract text from PDF page by page"""
    doc = fitz.open(pdf_path)
    pages = []
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()
        
        # Only include pages with meaningful text
        if text.strip():
            pages.append({
                "page_number": page_num + 1,
                "text": text.strip()
            })
    
    doc.close()
    return pages

res = extract_text_from_pdf()

<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


In [8]:
from tqdm import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter
def chunk_text(text, chunk_size=300, chunk_overlap=50):
    """Split text into chunks using RecursiveCharacterTextSplitter"""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    chunks = splitter.split_text(text)
    print(chunks)
    return chunks

def ingest_pdf(client, model, car_model="MG_Astor", pdf_filename="/Users/santhosh/Documents/study_projects/ringcentral_assessment/pdfs/astor_manual.pdf"):
    """Ingest a single PDF into Weaviate"""
    pdf_path = pdf_filename
    
    # Extract text from PDF
    pages = extract_text_from_pdf(pdf_path)
    print(f"   Extracted {len(pages)} pages")
    
    # Get collection
    collection = client.collections.get(COLLECTION_NAME)
    
    # Process each page
    total_chunks = 0
    chunk_index = 0
    
    with collection.batch.dynamic() as batch:
        for page in tqdm(pages, desc=f"   Chunking & embedding"):
            page_text = page["text"]
            page_num = page["page_number"]
            
            # Chunk the page text (renamed function call to be clear)
            text_chunks = chunk_text(page_text)
            
            # Process each chunk
            for chunk in text_chunks:  # Changed from chunk_text to chunk
                # Skip very small chunks
                if len(chunk.strip()) < 50:
                    continue
                
                # Generate embedding
                embedding = model.encode(chunk).tolist()
                
                # Create data object
                data_object = {
                    "text": chunk,  # Changed from chunk_text to chunk
                    "car_model": car_model,
                    "page_number": page_num,
                    "chunk_index": chunk_index,
                    "source_file": pdf_filename,
                    "chunking_strategy": "basic_recursive"
                }
                
                # Add to batch
                batch.add_object(
                    properties=data_object,
                    vector=embedding
                )
                
                chunk_index += 1
                total_chunks += 1
    
    print(f"✅ Inserted {total_chunks} chunks for {car_model}")
    return total_chunks

ingest_pdf(client, model)

   Extracted 257 pages


   Chunking & embedding:   1%|          | 3/257 [00:00<00:16, 15.86it/s]

["OWNER'S MANUAL"]
['u Do not use blower or infotainment system in engine \noff mode.\nu Ensure door, hood and tail gate are properly locked.\nu\n \n P \n In AT/CVT - Shift to\nbefore switching off ignition.']
['1\nWelcome to the world of MG!!!\nThank you for choosing MG Astor – India’s first Car with AI Inside\nThis manual will familiarize you with the operation and maintenance of your new vehicle. It will also provide you important safety information.', 'Please do read it carefully and follow the recommendations. This manual is like a permanent part of your car. It would go a long way in \nensuring a safe and trouble-free operation and running of your MG Astor.', 'In case of any query, please feel free to call our 24 hours Helpline at 1800 100 6464 or email us at: pulsehub@mgmotor.co.in\nFind your nearest MG authorized dealer by visiting www.mgmotor.co.in/tools/dealers\nSAFE and HAPPY DRIVING\nFrom\nTeam MG India']
['3\nContents\nPreface..........................................9\n\t

   Chunking & embedding:   2%|▏         | 5/257 [00:01<01:39,  2.53it/s]

['5\n•\t Anti-theft System ..................................118\n•\t Tailgate ....................................................120\n\t\nStarting and Stopping Engine..122\n•\t START/STOP Switch (Key Start)* ...122\n•\t START/STOP Switch  \n\t\n\t\n(Keyless Start)* .....................................123', '•\t Starting the Engine (Key Start)* ........124\n•\t Starting the Engine (Keyless Start) ...124\n•\t Precautions for Starting the Engine...126\n•\t Stopping the Engine...............................126\n\t\nEconomical and Environmental  \n\t\nDriving .......................................127', '•\t Running-in ..............................................127\n•\t Environment Protection .....................127\n•\t Economic Driving .................................127\n•\t Driving in Special Environment .........128\n•\t Check and Service ................................128', 'Catalytic Converter .................129\n\t\nFuel System ..............................131\n•\t Fuel 

...

   Chunking & embedding:  98%|█████████▊| 252/257 [00:14<00:00, 19.79it/s]

['ENGINE CONTROL SYSTEM\nDrive Belt\t\n\t \t Engine Oil (Replace)\t\n\t\n(Alternator, Power Steering and A/C Compressor)\nEngine Oil Filter (Replace)\t\n\t \t Cooling System Hose and Connections\t\n\t\nEngine Coolant\t\n\t \t Fuel Filter\t\n\t\nFuel Line and Connections\t\n\t \t Air Cleaner Element', 'Air Cleaner Element\t\n\t\nSpark Plugs & Spark Plug Wires\t\n\t \t \t\nCHASSIS AND BODY\nExhaust Pipe & Mountings\t\n\t \t Brake Fluid\t\n\t\nFront Brake Discs & Pads\t\n\t \t Rear Brake Drums & Linings\t\n\t .\nParking Brake\t\n\t \t Brake Line and Connections \t\n\t\n\t\n\t\n(Including Booster)\nManual Transaxle Fluid', '(Including Booster)\nManual Transaxle Fluid \t\n\t \t Steering Wheel & Linkage\t\n\t\nWheel Alignment\t\n\t \t Wheel Balancing\n(When required or as suggested by\t\n\t\n(When required or as suggested by \nMG Motor retailer) \t\n\t\nMG Motor retailer)\nPower Steering Fluid and Line\t\n\t \t Drive Shaft Boots', 'Drive Shaft Boots\t\n\t\nSafety Belts, Buckles & Anchorages\

   Chunking & embedding: 100%|██████████| 257/257 [00:14<00:00, 17.95it/s]


['260\nMAINTENANCE RECORD SHEET\n(Repair Category - Free Service / Paid Service / Running Repair / Accident Repair)\nRepair\nDate\nR.O. No.\nKms.\nRepair \nCategory\nDetails of Repair Done\nName of Servicing \nRetailer\nService Adv. \nSign.\nRetailer Stamp\nMaintenance Record Sheet']
['261\nMAINTENANCE RECORD SHEET\n(Repair Category - Free Service / Paid Service / Running Repair / Accident Repair)\nRepair\nDate\nR.O. No.\nKms.\nRepair \nCategory\nDetails of Repair Done\nName of Servicing \nRetailer\nService Adv. \nSign.\nRetailer Stamp\nMaintenance Record Sheet']
["VEHICLE RECORD SHEET\nMODEL\nCHASSIS NO.\nENGINE NO.\nKEY NO.\nDATE OF DELIVERY\nMILEAGE (KM)\nBATTERY\nTYRE\nMAKE\nSR NO./ BATCH CODE\nOWNER'S NAME & ADDRESS:\nShowroom:\nWorkshop:\nFor any feedback, please call Toll free No. 1800 100 6464\nADDRESS OF SELLING DEALER\nMAKE\nBATCH NUMBER\nTYRE LOCATION\nFR RH\nFR LH", 'MAKE\nBATCH NUMBER\nTYRE LOCATION\nFR RH\nFR LH\nRR RH\nRR LH']
["www.mgmotor.co.in\n1800 100 6464\nMG Motor

1698

In [12]:
print(res[43]['text'])

44
IMPORTANT
To avoid possible SRS issues, please do not 
press with excessive force or hit the airbag 
cover when operating the horn.
Horn
Press the horn button area on the steering 
wheel (as indicated by the arrow) to operate 
the horn.
Note: The vehicle horn switch location 
and the driver’s airbag are located in 
close proximity on the steering wheel. The 
illustration shows the position of the horn 
switches, please ensure that you press in 
this area to avoid any potential conflict 
with the operation of the airbag.
Mirrors
Door Mirrors
Note: Objects viewed in door mirrors may 
appear further away than they actually 
are.
Electric Door Mirror Glass 
Adjustment
•	
The mirror adjustment function will 
work with the START/STOP Switch in 
all modes, including OFF, ACC and ON/
RUNNING.
•	
Rotate the knob to select left (L) or right 
(R) rearview mirror.
Instruments and Controls


In [1]:
import yaml
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)
TOP_K_CHUNKS = 5
GROQ_MODEL = config["generator_model_params"]["model_name"]
EMBEDDING_MODEL = config["embedding_model"]
CHUNKING_STRATEGIES = config["chunking_strategies"]
DEFAULT_STRATEGY = "basic_recursive"

In [3]:
CHUNKING_STRATEGIES.keys()

dict_keys(['basic_recursive', 'sentence_based', 'semantic_based', 'semantic_based_v2'])

In [6]:
str = """
30
Cruise Control Indicator - 
Green/Yellow *
 If the cruise control function 
is enabled, the system will enter into the 
standby state and the indicator illuminates in 
yellow. When the cruise control system operates, 
this indicator illuminates green, indicating the 
cruise control system is activated. If a failure in the cruise control system is 
detected, the indicator will flash in yellow. Please contact an MG Authorised Dealer as 
soon as possible. Tyre Pressure Monitoring 
System (TPMS) Warning Lamp - 
Yellow *
 If this lamp illuminates, it indicates 
a tyre pressure is low, please check the tyre 
pressures. If this lamp flashes first and then remains on 
after a period of time, it indicates the system 
has a failure. Please contact an MG Authorised 
Dealer at the earliest opportunity. Stability Control/Traction 
Control System Warning Lamp 
- Yellow
 If this lamp illuminates, it indicates 
that the stability control/traction control 
system has a failure.
"""

In [7]:
len(str)

980

In [ ]:
    import json

    astor_manual_files = ['../feature_extraction/pdfs_for_landing_ai/astor_manual_1_90.json',
                            '../feature_extraction/pdfs_for_landing_ai/astor_manual_90_180.json',
                            '../feature_extraction/pdfs_for_landing_ai/astor_manual_181_266.json']

    tiago_manual_files = ['../feature_extraction/pdfs_for_landing_ai/tiago_manual_1_90.json',
                            '../feature_extraction/pdfs_for_landing_ai/tiago_manual_91_153.json']

    astor_chunks = []
    tiago_chunks = []

    last_page=0
    for part in astor_manual_files:

        with open(part, 'r') as file:
            data = json.load(file)
        
        for chunk in data['chunks']:
            # print(chunk['grounding']['page'])

            chunk['grounding']['page'] = chunk['grounding']['page'] + last_page + 1
            astor_chunks.append(chunk)
        
        last_page = data['chunks'][-1]['grounding']['page']
        print(last_page)
    print(len(astor_chunks))
    last_page=0
    for part in tiago_manual_files:

        with open(part, 'r') as file:
            data = json.load(file)
        
        for chunk in data['chunks']:
            # print(chunk['grounding']['page'])

            chunk['grounding']['page'] = chunk['grounding']['page'] + last_page + 1
            tiago_chunks.append(chunk)
        
        last_page = data['chunks'][-1]['grounding']['page']
        print(last_page)

    print(len(tiago_chunks))

90
180
266
2215
90
153
1091


In [21]:
print(len(astor_chunks),len(tiago_chunks))

2215 1091


In [22]:
astor_chunks[0]

{'markdown': '<a id=\'47209486-fb49-4e67-ab6b-b8c556c91b3e\'></a>\n\n<::logo: MG\nMORRIS GARAGES\nSince 1924\nThe logo features the letters "MG" within an octagonal frame, with "MORRIS GARAGES" and "Since 1924" to its right, all in white on a black background.::>',
 'type': 'logo',
 'id': '47209486-fb49-4e67-ab6b-b8c556c91b3e',
 'grounding': {'box': {'left': 0.07289601117372513,
   'top': 0.10545793175697327,
   'right': 0.30758294463157654,
   'bottom': 0.20162925124168396},
  'page': 1}}

In [30]:
unique = set()
for ele in astor_chunks:
    unique.add(ele['type'])
    if ele['type'] == 'table':
        print(ele)

{'markdown': "<a id='88cb0b03-90eb-4a1c-b3be-33e59eb15a29'></a>\n\n<table><thead><tr><th>Topic</th><th>Page</th></tr></thead><tbody><tr><td><b>Wheel Replacement</b></td><td><b>188</b></td></tr><tr><td><ul><li>Spare Wheel and Toolbox</li></ul></td><td>188</td></tr><tr><td><ul><li>Changing a Wheel</li></ul></td><td>189</td></tr></tbody></table>", 'type': 'table', 'id': '88cb0b03-90eb-4a1c-b3be-33e59eb15a29', 'grounding': {'box': {'left': 0.37885844707489014, 'top': 0.4811581075191498, 'right': 0.6368170976638794, 'bottom': 0.5689786672592163}, 'page': 8}}
{'markdown': '<a id=\'1fc8895b-e624-4796-aedb-4690f4c5a0f0\'></a>\n\n<table id="14-1">\n<tr><td id="14-2">Instruments and Controls</td><td id="14-3"></td></tr>\n<tr><td id="14-4">Instruments and Controls...................</td><td id="14-5">14</td></tr>\n<tr><td id="14-6">Information Centre ................................</td><td id="14-7">17</td></tr>\n<tr><td id="14-8">Warning Lights and Indicators..................</td><td id="14-9"

In [27]:
astor_chunks[0]

{'markdown': '<a id=\'47209486-fb49-4e67-ab6b-b8c556c91b3e\'></a>\n\n<::logo: MG\nMORRIS GARAGES\nSince 1924\nThe logo features the letters "MG" within an octagonal frame, with "MORRIS GARAGES" and "Since 1924" to its right, all in white on a black background.::>',
 'type': 'logo',
 'id': '47209486-fb49-4e67-ab6b-b8c556c91b3e',
 'grounding': {'box': {'left': 0.07289601117372513,
   'top': 0.10545793175697327,
   'right': 0.30758294463157654,
   'bottom': 0.20162925124168396},
  'page': 1}}

In [31]:
unique

{'attestation',
 'card',
 'figure',
 'logo',
 'marginalia',
 'scan_code',
 'table',
 'text'}